In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
credits= pd.read_csv("E:/DataScienceProject/archive/tmdb_5000_credits.csv")

In [ ]:
movies=pd.read_csv("E:/DataScienceProject/tmdb_5000_movies.csv")


movies.head(5)

In [ ]:
movies

In [ ]:
credits

In [ ]:
credits.shape

In [ ]:
movies.shape

In [ ]:
movies= movies.merge( credits, on ='title')

In [ ]:
movies.shape

In [ ]:
#will recoomendate on 7 criteria 
#genres
#id
#keyword
#title
#overview
#cast
#crew
movies.columns


In [ ]:
movies= movies[['genres','id', 'keywords','title', 'overview','cast', 'crew']]

In [ ]:
movies

In [ ]:
movies.shape

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace =True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast
def convert(obj) :
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
        return l

In [ ]:
movies.head()

In [ ]:
movies

In [ ]:
movies.iloc[0].keywords

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
movies

In [ ]:
movies

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies

In [ ]:
movies